# Mensa Daten

In [ ]:
import tarfile
from datetime import datetime

from bs4 import BeautifulSoup
import pandas as pd
import plotly.express as px
import spacy

from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

## Inhaltsverzeichnis
- [Vorbereitung](#Vorbereitung)
- [Plausibilitätsprüfung](#Plausibilitätsprüfung)
- [Aufteilung der Daten](#Aufteilung-der-Daten)
- [Analyse von Texten](#Analyse-von-Texten)
- [Transformation des Textes](#Transformation-des-Textes)
- [Regressionspipelines](#Regressionspipelines)
- [Evaluation](#Evaluation)
- [Übertragbarkeit auf zukünftige Ereignisse](#Übertragbarkeit-auf-zukünftige-Ereignisse)

## Vorbereitung

Laden der generellen Informationen:

In [ ]:
mensen_info: pd.DataFrame = pd.read_csv('mensen_info.csv', encoding='utf-8')
mensen_info[mensen_info['city'] == 'Ilmenau']

Einlesen des komprimierten Tarballs:

In [ ]:
def load_tar_gz(path, whitelist=None):
    with tarfile.open(path, 'r:gz', encoding='utf-8') as tar:
        for member in tar.getmembers():
            with tar.extractfile(member) as file:
                mensa_id, year, month, day = map(int, member.name[:-5].split('-'))
                if whitelist is not None and mensa_id not in whitelist:
                    continue

                day = datetime(year=year, month=month, day=day, hour=12)
                html = BeautifulSoup(file, 'html.parser')

                yield mensa_id, day, html

HTML parsen:

In [ ]:
def load_meals(html):
    for meal in html.find_all('div', {'class': 'rowMeal'}):
        name = meal.find('div', {'class': 'mealText'}).text
        name = name.strip()

        prices_str = meal.find('div', {'class': 'mealPreise'}).text
        if prices_str.startswith('je 100 gr. '):
            continue

        prices_str_without_eur = prices_str[:-2]
        prices = [
            float(p.replace(',', '.'))
            for p in prices_str_without_eur.split('/')
        ]

        yield name, prices

Konvertieren in DataFrame:

In [ ]:
# Parameter `whitelist` beachten!
mensen_data = pd.DataFrame(
    (
        [mensa_id, day, name, *prices]
        for mensa_id, day, html in load_tar_gz('mensen_html.tar.gz', whitelist=(46, 55))
        for name, prices in load_meals(html)
    ),
    columns=['mensa', 'date', 'meal', 'student', 'employee', 'guest']
)
mensen_data

Join mit Daten zu einzelnen Mensen:

In [ ]:
df = pd.merge(mensen_data, mensen_info, left_on='mensa', right_on='id')
df.drop(columns=['id'], inplace=True)

df

In [ ]:
df.info()

## Plausibilitätsprüfung

Gerichte an Feiertagen müssen fehlen:

In [ ]:
df[(datetime(2023, 10, 2) < df['date']) & (df['date'] < datetime(2023, 10, 5))]

Preisentwicklung über die Zeit:

In [ ]:
df['month'] = df['date'].map(lambda x: datetime(day=1, month=x.month, year=x.year, hour=12))
df_grouped = df.groupby(['month', 'name'])['student'].mean().reset_index()

fig = px.bar(df_grouped, x='month', y='student', color='name', barmode='group')
fig.update_xaxes(title_text='Monat')
fig.update_yaxes(title_text='Durchschnittlicher Preis', range=[2.5, 3.8])

fig

Einträge der NANOTeria fehlen in den ersten drei Quartalen:

[Cafeteria NANOteria wurde wiedereröffnet! - Studierendenwerk Thüringen (12.10.2023)](https://www.stw-thueringen.de/news/cafeteria-nanoteria-wurde-wiedereroeffnet!.html)

Weitere Ideen:

## Aufteilung der Daten

Trennen Sie die Daten in Trainings- und Testdaten im Verhältnis 3:1. Reduzieren Sie die Daten dann auf die Feature- und Zielvariablen.

In [ ]:
df_train, df_test = ..., ...

X_train, y_train = [], []
X_test, y_test = [], []

len(X_train), len(X_test)

Aufräumen:

In [ ]:
del mensen_data, mensen_info, df, df_train, df_test

## Analyse von Texten
SpaCy ist ein Python Paket, das vorgefertigte Modelle für die effiziente Verarbeitung von Text bereitstellt. Es bietet insbesondere Embeddings für Sätze oder Wortgruppen an:

In [ ]:
nlp = spacy.load('de_core_news_md')
nlp

Vektoren des Modells besitzen immer eine Länge von 300:

In [ ]:
embedding = nlp('Nudelauflauf "Griechische Art" mit Ratatouille und Reiskornnudeln').vector
len(embedding)

## Transformation des Textes

Teil der Pipeline muss das Umwandeln des Gerichts in einen Vektor sein. Verfassen Sie einen Transformator, der die Spalte `meal` in mehrere Spalten mit den einzelnen Vektorelementen umwandelt.

In [ ]:
class Embedding(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # TODO X['meal'] -> DataFrame mit Vektoren
        return X

Das Ergebnis des Transformator sollte dann ein DataFrame mit $300$ Spalten sein:

In [ ]:
Embedding().transform(X_train)

## Regressionspipelines

### Methode der kleinsten Quadrate
Erzeugen Sie eine Pipeline, die eine lineare Regression mit Hilfe der Klasse `LinearRegression` vornimmt.

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
ols_pipeline = Pipeline([
    ...
])

In [ ]:
ols_pipeline.fit(X_train, y_train)

Eventuell ist die Schätzung des Problems nicht linear vornehmbar. Mit Hilfe der Klasse `PolynomialFeatures` lassen sich Kombinationen aus Features erzeugen.

Beispiel: $\{ a, b \} \rightarrow \{ 1, a, b, a^2, a*b, b^2 \}$

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
pols_pipeline = Pipeline([
    ...
])

In [ ]:
pols_pipeline.fit(X_train, y_train)

### Entscheidungsbaum
Verwenden Sie die Klasse `DecisionTreeRegressor`, um einen Entscheidungsbaum zur Regression zu verwenden.

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
tree_pipeline = Pipeline([
    ...
])

In [ ]:
tree_pipeline.fit(X_train, y_train)

### Nächster Nachbar
Verwenden Sie die Klasse `KNeighborsRegressor`, um einen Nächsten-Nachbar-Algorithmus zur Regression zu verwenden.

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
nn_pipeline = Pipeline([
    ...
])

In [ ]:
nn_pipeline.fit(X_train, y_train)

### Deep Learning
Verwenden Sie die Klasse `MLPRegressor`, um ein neuronales Netz zur Regression zu verwenden.

In [ ]:
from sklearn.neural_network import MLPRegressor

In [ ]:
dnn_pipeline = Pipeline([
    ...
])

In [ ]:
dnn_pipeline.fit(X_train, y_train)

## Evaluation
*Mean Squared Error* (MSE) berechnet die durchschnittliche, quadrierte Abweichung der geschätzten von den tatsächlichen Werten.

$$ MSE(y, \hat{y}) = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y_i})^2 $$

Verwenden Sie MSE, um die Leistung der trainierten Regressionspipelines zu bewerten. Verwenden Sie dabei **nacheinander** Trainings- und Testdaten.

In [ ]:
all_pipelines = [
    ('ols', ols_pipeline),
    ('pols', pols_pipeline),
    ('tree', tree_pipeline),
    ('nn', nn_pipeline),
    ('dnn', dnn_pipeline)
]

In [ ]:
...

Bewerten Sie die Ergebnisse. Gehen Sie dabei auf Overfitting ein.

## Übertragbarkeit auf zukünftige Ereignisse
Erstellen Sie ein DataFrame, das den Essensplan des aktuellen Tages widerspiegelt.

In [ ]:
new_meals = pd.DataFrame({
    'date': [
        datetime.now()
        for _ in range(4)
    ],
    'meal': [
        'Tofusteak mit Spinat-Tomaten - Kokos - Gemüse mit Reis',
        'Bunte Bratwurstpfanne mit Gemüse und Nudeln',
        'Puten-Leberragout in Rotweinsoße an Kartoffelröstis und Krautsalat',
        'Nudelauflauf "Griechische Art" mit Ratatouille und Reiskornnudeln'
    ]
})
new_prices = pd.Series([1.95, 2.70, 3.10, 3.10])

Bewerten Sie erneut die Leistung der trainierten Regressionspipelines.

In [ ]:
...

Bewerten Sie die Modelle hinsichtlich ihrer Übertragbarkeit auf neue Daten. Diskutieren Sie Gründe, warum die Modelle eine schlechtere Leistung als auf den Testdaten zeigen.